In [21]:
import json
import math
import pickle
import random
import datetime
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

## 一.瓜子车源数据匹配

In [57]:
# coding=utf-8
import os
import re
import sys
import json
import math
import random
import datetime
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

# reload(sys)
# sys.setdefaultencoding('utf8')

ENCODING = 'utf-8'
# PRODUCE_DB_ADDR_OUTTER = '10.45.138.200'
# PRODUCE_DB_USER = 'pingjia'
# PRODUCE_DB_PASSWD = 'De32wsxC'

PRODUCE_DB_ADDR_OUTTER = '101.201.143.74'
PRODUCE_DB_USER = 'leidengjun'
PRODUCE_DB_PASSWD = 'ldj_DEV_~!'

PRODUCE_PINGJIA_ENGINE = 'mysql+pymysql://'+PRODUCE_DB_USER+':'+PRODUCE_DB_PASSWD+'@'+PRODUCE_DB_ADDR_OUTTER+'/pingjia?charset=utf8'
PRODUCE_CHINA_USED_CAR_ESTIMATE_ENGINE = 'mysql+pymysql://' + PRODUCE_DB_USER + ':' + PRODUCE_DB_PASSWD + '@' + PRODUCE_DB_ADDR_OUTTER + '/china_used_car_estimate?charset=utf8'
engine = create_engine(PRODUCE_CHINA_USED_CAR_ESTIMATE_ENGINE)

yesterday = datetime.datetime.now() - datetime.timedelta(days=1)
start = str(yesterday.year) + '-' + str(yesterday.month) + '-' + str(yesterday.day) + ' 00:00:00'
today = datetime.datetime.now()
end = str(today.year) + '-' + str(today.month) + '-' + str(today.day) + ' 00:00:00'

# query_sql = 'select * from crawler_car_source ' \
#             'where domain = \'guazi.com\' and create_time >= \''+start+'\' and  create_time < \''+end+'\' limit 1000'
query_sql = 'select * from crawler_car_source ' \
            'where domain = \'baixing.com\' or domain = \'58.com\' '
# query_sql = 'select source_id,detail_name,year,month,mile,price,city,domain,transfer_owner from crawler_car_source'
origin_train = pd.read_sql_query(query_sql, engine)
origin_train.to_csv('./origin_train.csv', index=False)
print('finish query, num:'+str(len(origin_train)))

finish query, num:66302


### 一.车源更新脚本

In [58]:
# coding=utf-8
import os
import re
import json
import math
import pickle
import random
import datetime
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

def process_price_bn(df):
    if df['price_bn'] == '暂无报价':
        return np.NAN
    temp = re.findall(r'万',df['price_bn'])
    if len(temp) > 1:
        text = df['price_bn'].split('~')[1]
    else:
        text = df['price_bn']
    text = text.replace('万','')
    return float(text)

def delete_str_useless(df, column_name):
    """
    删除没用的字符
    """
    text = df[column_name]
    text = text.lower()
    text = text.replace('ⅰ', 'i')
    text = text.replace('ⅱ', 'ii')
    text = text.replace('ⅲ', 'iii')
    text = text.replace('[', '')
    text = text.replace(']', '')
    text = text.replace('【', '')
    text = text.replace('】', '')
    text = text.replace('（', '')
    text = text.replace('）', '')
    text = text.replace('(', '')
    text = text.replace(')', '')
    text = text.replace('_', '')
    text = text.replace('-', '')
    text = text.replace('+', '')
    text = text.replace('—', '')
    text = text.replace('/', '')
    text = text.replace('“', '')
    text = text.replace('”', '')
    text = text.replace('!', '')
    text = text.replace('。', '')
    text = text.replace('＞', '')
    text = text.replace('·', '')
    text = text.replace('・', '')
    text = text.replace('》', '')
    text = text.replace('！', '')
    text = text.replace('／', '')
    text = text.replace('’', '')
    text = text.replace('－', '')
    text = text.replace('•', '')
    text = text.replace('×', '')
    text = text.replace('《', '')
    text = text.replace('＿', '')
    text = text.replace('®', '')
    text = text.replace('─', '')
    text = text.replace('、', '')
    text = text.replace(' ', '') 
    return text

def process_url(df):
    return df['url'].replace('https://','')

def process_eval_pirce(df):
    random_price = random.randint(int(-df['price']*0.95*0.03*10000),int(df['price']*0.95*0.03*10000))
    if (df['price'] * 0.95 + random_price/10000) < 0.5:
        return df['price']
    return float('%.2f' % (df['price'] * 0.95 + random_price/10000))

def process_gpj_index(df):
    k = 2
    b = abs(df['price'] * 0.95 - df['eval_price']) / df['eval_price']
    return pd.Series([float('%.2f' % (10 * (math.e ** (-k*b)))),int((10 * (math.e ** (-k*b)))*100000000)])

def process_thumbnail(df):
    url = []
    thumbnail = json.loads(df['thumbnail'])
    if df['domain'] == 'guazi.com':
        if len(thumbnail) < 10:
            return pd.Series([np.NaN,np.NaN])
        url.append(thumbnail[0]['images'][0]['image'])
        url.append(thumbnail[1]['images'][0]['image'])
        url.append(thumbnail[5]['images'][0]['image'])
        url.append(thumbnail[6]['images'][0]['image'])
        url.append(thumbnail[7]['images'][0]['image'])
        url.append(thumbnail[8]['images'][0]['image'])
        url.append(thumbnail[9]['images'][0]['image'])
        url.append(thumbnail[10]['images'][0]['image'])
        return pd.Series([thumbnail[0]['images'][0]['image'],url])
    else:
        url.append(thumbnail[0]['images'][0]['image'])
        url.extend(thumbnail[1]['images'][0]['image'].split(' '))
        return pd.Series([thumbnail[0]['images'][0]['image'],url])
    return pd.Series([np.NaN,np.NaN])

def add_brand(brand):
    brand = brand.append(pd.DataFrame([['昌河','79'],['吉利','25'],['传祺','82'],['swm斯威','269'],['斯威','269'],['猎豹','78'],
                                      ['力帆','80'],['五菱','114'],['野马','111'],['mg','20'],['英致','192'],['汉腾','267'],
                                      ['吉奥','108'],['康迪','219'],['夏利','110'],['莲花','89'],['北汽','154']],columns=['brand_name','brand_slug']),sort=False).reset_index(drop=True)
    return brand

def find_brand(df):
    for i in range(0,len(brand)):
        if brand.loc[i,'brand_name'] == 'mg':
            match = re.findall('^mg',df['detail_name'])
            if len(match) > 0:
                return pd.Series([brand.loc[i,'brand_name'],brand.loc[i,'brand_slug']])
        else:           
            match = re.findall(brand.loc[i,'brand_name'],df['detail_name'])
            if len(match) > 0:
                return pd.Series([brand.loc[i,'brand_name'],brand.loc[i,'brand_slug']])
    return pd.Series([np.NaN,np.NaN])

def process_online_year(df):
    temp = re.findall('(\d\d\d\d)款',df['detail_name'])
    if len(temp) == 0:
        return np.NaN
    return temp[0]

def process_volume(df):
    temp = re.findall('(\d\.\d)',df['detail_name'])
    if len(temp) == 0:
        return np.NaN
    return temp[0]

def process_control(df):
    text = df['detail_name']
    text = text.lower()
    temp = re.findall('自动|手自|amt|无级|cvt|双离合|dct|dsg',text)
    if len(temp) > 0:
        return '自动'
    temp = re.findall('手动',text)
    if len(temp) > 0:
        return '手动'
    return np.NaN

def cal_brand_count(df):
    return len(df['brand_name'])

def cal_cos_similar(df,y_vector):
    x_vector = eval(df['fill_vector'])
    y_vector = eval(y_vector)
    return np.dot(x_vector,y_vector) / (np.linalg.norm(x_vector,ord=2)*np.linalg.norm(y_vector,ord=2) )

def match_detail(df):
    print(df['brand_slug'],df['online_year'],df['control'],df['volume'])
    if (type(df['control']) != str) & math.isnan(float(df['volume'])):
        temp_detail = car_autohome_all.loc[(car_autohome_all['brand_slug'] == int(df['brand_slug']))&(car_autohome_all['online_year'] == int(df['online_year'])),:].reset_index(drop=True)
    elif (type(df['control']) != str) & (not math.isnan(float(df['volume']))):
        temp_detail = car_autohome_all.loc[(car_autohome_all['brand_slug'] == int(df['brand_slug']))&(car_autohome_all['online_year'] == int(df['online_year']))&(car_autohome_all['volume'] == float(df['volume'])),:].reset_index(drop=True)
    elif (type(df['control']) == str) & (math.isnan(float(df['volume']))):
        temp_detail = car_autohome_all.loc[(car_autohome_all['brand_slug'] == int(df['brand_slug']))&(car_autohome_all['online_year'] == int(df['online_year']))&(car_autohome_all['control'] == df['control']),:].reset_index(drop=True)
    else:
        temp_detail = car_autohome_all.loc[(car_autohome_all['brand_slug'] == int(df['brand_slug']))&(car_autohome_all['online_year'] == int(df['online_year']))&(car_autohome_all['control'] == df['control'])&(car_autohome_all['volume'] == float(df['volume'])),:].reset_index(drop=True)
    
    if len(temp_detail) == 0:
        temp_detail = car_autohome_all.loc[(car_autohome_all['brand_slug'] == int(df['brand_slug']))&(car_autohome_all['online_year'] == int(df['online_year'])),:].reset_index(drop=True)
        if len(temp_detail) == 0:
            return pd.Series([np.NaN,np.NaN,np.NaN,np.NaN,np.NaN])
    temp_detail['cos_similar'] = temp_detail.apply(cal_cos_similar,args=(df['fill_vector'],),axis=1)
    temp_detail = temp_detail.loc[temp_detail.groupby(['brand_slug']).cos_similar.idxmax(), :]
    result = temp_detail.loc[temp_detail.groupby(['brand_slug']).cos_similar.idxmax(), ['model_slug','model_name','detail_slug','detail_name','cos_similar']].values[0]
    return pd.Series(result)

#########################################step1
car_autohome_all = pd.read_csv('./car_autohome_cos_vector.csv')

pcm = pd.read_csv('./province_city_map.csv')
pcm = pcm.loc[:,['city','province']]

gpj_model = pd.read_csv('./combine_model.csv',low_memory=False)
gpj_model = gpj_model.loc[:,['slug','parent','classified_slug','popular']]
gpj_model = gpj_model.rename(columns={'slug':'model_slug','parent':'brand_slug','classified_slug':'body_style'})

gpj_detail = pd.read_csv('./combine_detail.csv',low_memory=False)
gpj_detail = gpj_detail.loc[:,['global_slug','volume','control','car_autohome_detail_id','detail_model_slug']]
gpj_detail = gpj_detail.rename(columns={'global_slug':'model_slug','car_autohome_detail_id':'detail_slug'})
gpj_detail = gpj_detail.merge(gpj_model,how='left',on=['model_slug'])
gpj_detail = gpj_detail.drop_duplicates(['detail_slug','volume','control']).reset_index(drop=True)

##########################################step2
origin_train = pd.read_csv('./origin_train.csv')

origin_train = origin_train.loc[(origin_train['detail_name'].notnull())&(origin_train['year'].notnull()),:].reset_index(drop=True)
origin_train['online_year'] = origin_train.apply(process_online_year,axis=1)
origin_train = origin_train.loc[(origin_train['online_year'].notnull()),:].reset_index(drop=True)
origin_train['volume'] = origin_train.apply(process_volume,axis=1)
origin_train['control'] = origin_train.apply(process_control,axis=1)
origin_train['title'] = origin_train['detail_name']
origin_train['detail_name'] = origin_train.apply(delete_str_useless,args=('detail_name',),axis=1)

brand = car_autohome_all.loc[:,['brand_name','brand_slug']].drop_duplicates(['brand_name','brand_slug']).reset_index(drop=True)
brand = add_brand(brand)
brand['name_count'] = brand.apply(cal_brand_count,axis=1)
brand = brand.sort_values(by=['name_count'],ascending=False).reset_index(drop=True)
model = car_autohome_all.loc[:,['brand_name','brand_slug','model_name','model_slug']].drop_duplicates(['brand_name','brand_slug','model_name','model_slug']).reset_index(drop=True)
origin_train[['brand_name','brand_slug']] = origin_train.apply(find_brand,axis=1)
origin_train = origin_train.loc[(origin_train['brand_slug'].notnull()),:].reset_index(drop=True)
# origin_train.info()

f = open('./word_index.txt', 'r', encoding='UTF-8')
temp = f.read()
word_index = eval(temp)

with open('./tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

test_final = []
for i in range(0, len(origin_train)):
    temp = []
    x = origin_train['detail_name'][i]
    temp.extend(x)
    test_final.append(temp)
test_sequences = tokenizer.texts_to_sequences(test_final)

fill_zero = [0 for i in range(0,len(word_index))]

for i in range(0,len(origin_train)):  
    text = fill_zero.copy()
    for j in test_sequences[i]:
        text[j-1] = text[j-1] + 1
    origin_train.loc[i,'fill_vector'] = str(text)
origin_train = origin_train.rename(columns={'detail_name':'origin_name'})

origin_train[['model_slug','model_name','detail_slug','detail_name','cos_similar']] = origin_train.apply(match_detail,axis=1) 
origin_train = origin_train.loc[(origin_train['cos_similar'] >= 0.82),:].reset_index(drop=True)
origin_train = origin_train.drop(['fill_vector','cos_similar','control','volume','online_year'],axis=1)

fill_columns = car_autohome_all.loc[:,['detail_slug','online_year','price_bn','volume','control']]
cur_time = datetime.datetime.now()
car_source = origin_train.drop(['id','car_application','labels','title','brand_slug','model_slug','brand_name','model_name','detail_name'],axis=1)
car_source['pub_time'] = car_source['create_time']
car_source['status'] = 'sale'
car_source.loc[(car_source['domain'].isin(['guazi.com','xin.com'])),'source_type'] = 'cpersonal'
car_source.loc[(car_source['domain'].isin(['58.com','baixing.com'])),'source_type'] = 'personal'
car_source['process_status'] = 'S'
car_source['global_sibling'] = 0
car_source['city_sibling'] = 0
car_source['dealer_sibling'] = 0
car_source['domain_sibling'] = 0
car_source['qs_tags'] = '质保'
car_source['expired_at'] = datetime.datetime.now() + datetime.timedelta(days=10)
car_source['domain_sibling'] = 0
car_source['created_on'] = cur_time 
car_source['cs_clean'] = 0
car_source = car_source.merge(pcm, how='left', on=['city'])
car_source = car_source.merge(fill_columns, how='left', on=['detail_slug'])
car_source = car_source.merge(gpj_detail,how='left',on=['detail_slug','volume','control'])
car_source = car_source.rename(columns={'origin_name':'title','detail_model_slug':'model_detail_slug','popular':'popularity'})
car_source = car_source.drop(['create_time','detail_slug'],axis=1)
car_source = car_source.loc[(car_source['model_detail_slug'].notnull()),:].reset_index(drop=True)
car_source = car_source.drop_duplicates(['url'])
car_source['url'] = car_source.apply(process_url, axis=1)
car_source['eval_price'] = car_source.apply(process_eval_pirce, axis=1)
car_source[['gpj_index','rank']] = car_source.apply(process_gpj_index, axis=1)
car_source[['thumbnail','car_image']] = car_source.apply(process_thumbnail, axis=1)
car_source = car_source.loc[(car_source['car_image'].notnull()),:].reset_index(drop=True)
car_source = car_source.sort_values(by=['url','pub_time'],ascending=[True,False]).reset_index(drop=True)
car_source = car_source.drop_duplicates(['url']).reset_index(drop=True)
car_source.to_csv('./finish_match.csv',index=False)

25 2016 自动 1.3
25 2016 自动 1.3
1 2017 自动 1.4
75 2017 自动 1.5
25 2017 自动 1.3
1 2016 自动 1.8
76 2015 手动 1.6
95 2016 自动 1.6
14 2015 自动 1.5
38 2017 自动 1.5
3 2017 自动 1.5
82 2017 自动 1.5
38 2016 自动 1.5
62 2016 自动 2.0
14 2017 自动 1.5
25 2017 自动 1.3
1 2016 自动 1.4
88 2016 自动 2.0
58 2010 自动 2.0
76 2017 自动 1.5
12 2016 nan 1.6
1 2014 自动 1.8
76 2017 nan nan
120 2017 手动 1.5
19 2018 自动 1.5
14 2016 自动 1.8
38 2016 自动 1.5
1 2017 自动 1.6
19 2018 自动 1.5
181 2017 自动 2.0
14 2010 自动 2.4
63 2016 自动 2.0
25 2017 自动 1.5
120 2016 手动 1.5
120 2017 手动 1.5
181 2018 自动 1.5
33 2012 自动 2.0
25 2016 自动 1.8
95 2015 自动 1.8
14 2016 自动 1.5
1 2017 自动 1.4
95 2015 手动 2.0
14 2014 自动 1.5
14 2009 自动 2.0
19 2016 自动 1.5
14 2017 自动 1.5
3 2016 自动 1.6
95 2015 自动 2.0
14 2017 自动 1.8
12 2010 自动 2.4
14 2016 自动 1.5
114 2016 手动 1.5
14 2012 自动 2.0
12 2016 nan 1.6
58 2016 自动 2.0
63 2016 自动 2.0
14 2016 自动 1.5
71 2016 自动 2.0
1 2011 自动 1.8
15 2009 自动 2.0
3 2011 自动 1.8
12 2010 自动 2.4
36 2014 nan nan
14 2014 自动 1.5
14 2007 自动 2.4
95 2009 手动 2.0
75 2015 手动

63 2018 自动 1.6
3 2016 自动 1.6
14 2016 自动 1.5
14 2009 自动 2.0
3 2011 自动 1.8
14 2016 自动 1.5
12 2015 自动 2.0
1 2012 手动 1.6
3 2011 自动 2.7
71 2015 手动 1.5
1 2007 手动 1.6
110 2012 nan 1.0
181 2015 手动 1.5
12 2013 nan 2.0
110 2010 手动 1.0
3 2016 自动 1.6
12 2009 自动 2.0
71 2013 自动 2.4
15 2014 nan nan
3 2011 自动 1.8
71 2016 自动 1.5
1 2015 手动 1.4
22 2009 nan 1.6
67 2012 手动 1.6
36 2013 nan nan
63 2004 手动 2.4
110 2010 手动 1.0
14 2015 自动 2.4
38 2006 手动 2.0
95 2011 手动 1.6
86 2015 手动 1.5
12 2013 nan 2.0
75 2010 手动 1.0
3 2015 自动 2.5
19 2017 自动 1.5
1 2016 自动 1.4
12 2006 手动 2.0
1 2008 自动 1.6
49 2007 nan nan
49 2009 nan nan
81 2015 手动 1.5
63 2016 自动 1.6
14 2004 自动 2.0
181 2014 手动 1.5
68 2003 手动 2.4
1 2009 自动 1.8
38 2004 自动 1.6
1 2011 自动 1.4
12 2011 nan 1.6
14 2009 自动 2.4
14 2012 自动 2.4
62 2016 手动 1.6
12 2009 自动 2.0
33 2004 自动 2.8
38 2008 自动 3.0
1 2018 自动 1.4
77 2014 手动 1.5
20 2017 自动 1.5
58 2016 自动 2.0
95 2016 自动 2.0
1 2018 自动 1.4
38 2015 自动 1.5
181 2013 手动 2.0
63 2017 自动 1.5
1 2016 自动 1.4
108 2014 手动 1.5
1 2018 nan

12 2009 自动 2.0
84 2009 手动 2.0
63 1997 nan nan
15 2010 自动 2.0
15 2012 nan nan
38 2017 nan nan
1 2009 nan 2.0
3 2012 自动 2.5
72 2012 自动 2.0
70 2013 自动 2.0
82 2017 自动 1.3
1 2016 nan 1.6
8 2012 nan 1.8
3 2017 自动 1.2
14 2016 自动 1.8
36 2015 nan nan
67 2013 手动 1.6
82 2017 自动 1.5
63 2013 自动 1.6
14 2009 自动 2.0
52 2016 nan 2.0
14 2010 自动 2.0
58 2009 自动 1.6
33 2013 nan nan
36 2018 nan 2.0
25 2017 自动 1.3
51 2017 自动 2.0
14 2017 自动 1.5
142 2016 nan 1.5
1 2017 nan 1.8
38 2008 自动 3.0
65 2007 nan nan
12 2005 自动 2.0
12 2011 手动 1.4
1 2004 手动 1.8
82 2017 自动 2.0
8 2016 自动 1.5
1 2004 手动 1.8
103 2011 手动 1.3
1 2010 手动 1.6
1 2012 自动 1.8
1 2013 自动 1.6
103 2011 手动 1.3
77 2014 手动 1.5
14 2009 自动 2.0
1 2006 手动 1.6
82 2017 自动 1.5
25 2017 nan 1.5
15 2015 自动 2.0
25 2016 自动 1.8
12 2016 自动 1.4
58 2017 nan nan
12 2017 自动 1.4
38 2018 自动 1.0
20 2017 自动 1.5
1 2019 自动 2.0
81 2018 自动 1.5
25 2016 自动 1.8
20 2017 自动 1.5
1 2017 自动 1.6
12 2016 nan 1.6
82 2017 自动 1.3
25 2017 自动 1.3
1 2019 自动 2.0
15 2013 自动 2.0
3 2017 自动 1.2
82 2017 

14 2015 自动 1.8
142 2016 nan 1.5
12 2017 自动 1.4
84 2017 手动 1.5
63 2015 自动 1.5
58 2016 自动 2.0
1 2011 自动 1.8
19 2018 自动 1.6
12 2016 nan 1.6
75 2017 自动 1.5
120 2016 手动 1.5
1 2013 自动 1.6
3 2012 自动 2.0
3 2013 自动 2.0
1 2009 自动 1.8
82 2014 自动 2.0
3 2011 自动 2.4
14 2014 自动 1.8
15 2014 自动 2.0
12 2016 nan 1.6
8 2011 nan 1.8
62 2015 手动 1.4
122 2017 自动 2.0
71 2015 手动 1.5
130 2013 自动 2.2
86 2011 手动 1.6
81 2010 手动 1.5
1 2010 自动 1.8
14 2016 自动 1.8
12 2011 nan 1.6
14 2014 自动 1.5
36 2015 nan 2.0
71 2013 手动 1.2
62 2010 nan 1.6
14 2016 自动 2.4
77 2012 手动 1.5
53 2009 手动 0.8
81 2014 手动 1.5
14 2008 自动 2.4
53 2016 手动 1.5
24 2006 手动 1.6
1 2011 nan 3.0
13 2014 手动 1.6
1 2012 自动 2.0
1 2011 自动 1.8
3 2008 自动 1.8
1 2002 手动 1.6
1 2010 手动 1.6
14 2011 自动 3.5
70 2011 nan 2.0
1 2016 自动 1.6
15 2009 nan nan
63 2007 手动 2.4
15 2015 自动 2.0
33 2013 nan 2.0
1 2001 自动 1.6
75 2011 nan 1.5
75 2015 手动 1.5
95 2011 手动 1.6
77 2010 手动 2.0
63 2008 手动 1.6
1 2012 手动 1.6
63 2016 自动 2.0
110 2010 手动 1.0
3 2017 自动 1.2
38 2010 自动 2.0
24 2009 手动 

12 2016 自动 1.4
71 2016 nan 1.5
120 2019 nan 1.5
3 2017 自动 1.5
120 2018 自动 1.5
1 2018 nan 1.5
81 2018 自动 1.5
1 2015 自动 1.8
120 2019 nan 1.5
14 2016 自动 1.5
3 2018 自动 1.2
1 2017 自动 1.4
63 2016 自动 2.0
25 2016 自动 1.8
120 2017 手动 1.5
38 2016 自动 1.5
14 2016 自动 1.5
63 2016 自动 1.6
1 2017 自动 1.5
3 2017 自动 1.5
12 2017 自动 1.8
25 2017 自动 1.5
181 2017 自动 2.0
14 2016 自动 1.5
71 2016 自动 1.5
81 2017 自动 1.5
71 2016 自动 1.5
8 2017 nan 1.6
20 2017 自动 1.5
19 2018 自动 1.6
14 2015 自动 1.5
3 2017 自动 1.5
12 2016 nan 1.6
71 2016 自动 1.5
76 2017 nan nan
14 2016 自动 1.5
12 2017 自动 1.8
84 2017 手动 1.5
33 2012 自动 2.0
20 2018 自动 1.5
76 2017 自动 1.5
14 2017 自动 1.5
58 2016 自动 2.0
71 2016 自动 2.0
1 2016 自动 1.4
12 2016 nan 1.6
71 2016 自动 1.5
25 2017 自动 1.3
38 2016 自动 1.5
25 2018 自动 1.4
71 2015 手动 1.5
15 2013 自动 2.0
20 2017 自动 1.5
75 2017 自动 1.5
63 2016 自动 2.0
63 2013 自动 1.6
103 2011 手动 1.3
1 2014 自动 1.8
75 2013 自动 1.5
1 2018 nan 1.5
26 2007 手动 1.1
14 2017 手动 1.0
63 2015 自动 1.5
1 2016 自动 1.4
1 2018 nan 1.5
58 2016 自动 2.0
3 2015 自

...

1 2006 手动 1.8
3 2018 自动 2.0
3 2009 自动 1.8
181 2018 自动 1.5
14 2012 自动 1.5
71 2014 自动 1.4
14 2012 自动 2.0
14 2016 自动 1.8
82 2017 自动 1.5
38 2018 自动 1.3
14 2016 自动 1.5
36 2013 自动 1.8
181 2017 自动 2.0
62 2011 手动 1.6
75 2017 自动 1.5
63 2014 自动 2.0
181 2017 自动 1.5
63 2018 自动 1.6
14 2017 自动 2.0
75 2017 自动 1.5
14 2009 自动 2.4
14 2015 自动 2.0
47 2014 nan 2.0
1 2015 自动 1.6
1 2017 自动 1.4
1 2015 nan 1.4
63 2016 自动 1.6
81 2018 自动 1.5
181 2018 nan 1.5
63 2011 自动 1.6
14 2008 自动 2.0
3 2012 自动 2.5
181 2018 nan 1.5
3 2012 自动 2.5
36 2013 自动 1.8
1 2017 自动 1.4
1 2016 自动 1.6
12 2015 自动 1.6
12 2011 nan 2.0
36 2013 自动 1.8
38 2012 自动 1.6
1 2013 自动 1.6
14 2008 自动 2.0
63 2011 自动 1.6
63 2013 自动 2.0
3 2012 自动 2.5
181 2018 nan 1.5
1 2011 自动 2.0
10 2015 nan 1.2
14 2008 自动 2.0
1 2011 自动 2.0
36 2013 自动 1.8
63 2011 自动 1.6
10 2015 nan 1.2
10 2015 nan 1.2
14 2008 自动 2.0
3 2012 自动 2.5
63 2013 自动 2.0
1 2011 自动 2.0
36 2013 自动 1.8
63 2011 自动 1.6
10 2015 nan 1.2
36 2013 自动 1.8
63 2013 自动 2.0
3 2012 自动 2.5
1 2011 自动 2.0
63 2013 自动 2

71 2013 手动 1.4
33 2012 自动 2.0
8 2013 nan 1.8
38 2017 自动 1.5
12 2007 手动 2.0
67 2016 自动 1.4
14 2016 自动 1.8
63 2011 自动 1.6
38 2017 自动 1.5
1 2011 自动 2.0
38 2015 自动 1.5
53 2010 手动 1.4
38 2017 自动 1.5
15 2013 自动 2.0
63 2018 自动 1.6
36 2013 自动 1.8
82 2017 自动 1.3
14 2016 自动 2.0
38 2017 自动 1.5
38 2017 自动 1.5
33 2012 自动 2.0
13 2016 自动 1.6
13 2014 手动 1.6
14 2008 自动 2.0
3 2013 手动 2.0
1 2011 自动 2.0
38 2017 自动 1.5
1 2017 自动 1.4
38 2017 自动 1.5
3 2017 自动 1.2
1 2011 自动 2.0
14 2008 自动 2.0
13 2016 自动 1.6
63 2018 自动 1.6
33 2012 自动 2.0
38 2017 自动 1.5
38 2012 自动 1.6
3 2013 手动 2.0
36 2013 自动 1.8
3 2017 自动 1.2
8 2017 自动 1.5
38 2017 自动 1.5
25 2016 自动 1.5
14 2012 自动 1.5
13 2016 自动 1.6
62 2017 nan 1.4
25 2018 自动 1.4
38 2017 自动 1.5
63 2018 自动 1.6
38 2015 自动 1.5
3 2013 手动 2.0
1 2017 自动 1.4
36 2013 自动 1.8
1 2011 自动 2.0
14 2008 自动 2.0
1 2018 自动 1.8
25 2016 自动 1.5
33 2012 自动 2.0
38 2017 自动 1.5
25 2016 自动 1.5
38 2017 自动 1.5
3 2017 自动 1.2
3 2013 手动 2.0
3 2017 自动 1.2
12 2007 手动 2.0
1 2011 自动 2.0
1 2017 自动 1.4
75 2018 自动 1

63 2011 自动 1.6
3 2009 自动 1.8
3 2018 自动 2.0
47 2014 nan 2.0
3 2012 自动 2.5
3 2014 自动 1.6
63 2014 自动 2.0
3 2009 自动 1.8
1 2018 自动 1.5
1 2018 自动 1.5
19 2012 自动 2.4
3 2014 自动 1.6
47 2014 nan 2.0
3 2009 自动 1.8
19 2012 自动 2.4
63 2011 自动 1.6
3 2009 自动 1.8
3 2009 自动 1.8
3 2009 自动 1.8
3 2009 自动 1.8
3 2009 自动 1.8
3 2009 自动 1.8
181 2018 nan 1.5
3 2009 自动 1.8
3 2009 自动 1.8
3 2009 自动 1.8
14 2016 自动 1.8
72 2007 nan 1.6
62 2013 自动 1.6
1 2005 手动 1.8
33 2005 自动 3.0
12 2004 nan 2.7
38 2014 自动 1.4
22 2010 手动 1.5
26 2008 手动 1.1
38 2011 自动 1.6
1 2009 自动 2.0
14 2016 自动 2.4
14 2014 自动 2.4
26 2016 自动 1.6
1 2007 手动 1.6
1 2009 nan 1.8
181 2015 自动 1.5
53 2013 手动 1.3
33 2005 手动 1.8
3 2013 自动 1.6
63 2006 手动 1.6
1 2018 自动 1.5
75 2013 自动 1.5
1 2010 手动 1.6
120 2015 手动 1.8
75 2007 手动 1.5
63 2010 自动 2.5
14 2012 自动 1.8
120 2015 手动 1.8
33 2010 自动 2.4
72 2007 手动 1.6
1 2008 手动 1.6
1 2006 手动 1.6
12 2009 nan 1.6
1 2008 手动 1.6
26 2011 手动 1.0
63 2008 自动 2.5
12 2011 nan 2.0
25 2016 手动 1.5
90 2006 手动 2.4
1 2012 自动 1.8
1 2013 手动 1.

36 2013 自动 1.8
25 2008 手动 1.3
63 2011 自动 1.6
25 2016 自动 1.5
1 2018 自动 1.8
12 2011 nan 2.0
25 2016 自动 1.5
63 2013 自动 2.0
63 2016 自动 1.6
63 2011 自动 1.6
13 2016 自动 1.6
1 2018 自动 1.8
38 2005 自动 3.0
71 2017 自动 1.5
26 2011 nan 1.6
110 2010 手动 1.0
25 2012 手动 1.5
19 2018 自动 1.5
71 2010 手动 1.4
14 2005 自动 2.4
25 2010 手动 1.8
13 2010 nan 1.6
25 2016 自动 1.8
76 2012 手动 1.5
14 2008 自动 2.0
86 2005 手动 1.6
62 2006 自动 2.0
12 2011 nan 2.0
63 2010 自动 2.0
71 2017 nan 1.5
38 2017 自动 1.5
14 2016 自动 2.0
20 2017 自动 1.5
114 2010 手动 1.2
1 2017 自动 1.6
62 2016 自动 1.6
19 2018 自动 1.5
76 2010 自动 1.5
25 2016 自动 1.5
94 2017 自动 1.8
1 2017 自动 1.4
38 2017 自动 1.5
71 2018 自动 1.5
38 2017 自动 1.5
12 2010 手动 1.4
71 2017 nan 1.5
122 2018 自动 2.0
110 2011 手动 1.5
20 2017 自动 1.5
72 2011 手动 1.6
19 2018 自动 1.3
15 2014 自动 2.0
71 2012 手动 1.2
1 2012 自动 3.0
58 2016 自动 2.0
67 2017 自动 1.6
10 2015 nan 1.2
181 2017 nan 2.0
1 2012 自动 1.4
14 2008 自动 2.0
25 2016 自动 1.5
25 2016 自动 1.3
120 2016 手动 1.5
33 2009 自动 2.4
76 2011 手动 1.5
76 2010 手动 1.5
1 

94 2010 手动 1.3


In [59]:
# coding=utf-8
import os
import re
import json
import math
import random
import datetime
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

ENCODING = 'utf-8'
PRODUCE_DB_ADDR_OUTTER = '10.45.138.200'
PRODUCE_DB_USER = 'pingjia'
PRODUCE_DB_PASSWD = 'De32wsxC'
PRODUCE_PINGJIA_ENGINE = 'mysql+pymysql://'+PRODUCE_DB_USER+':'+PRODUCE_DB_PASSWD+'@'+PRODUCE_DB_ADDR_OUTTER+'/pingjia?charset=utf8'

engine = create_engine(PRODUCE_PINGJIA_ENGINE, encoding=ENCODING)  

car_source = pd.read_csv('./finish_match.csv')
with engine.begin() as con:
    insert_car_source = car_source.drop(['source_id','online_year','car_image','description','conditions','contact','region','company_name',
                'maintenance','insurance_money','quality_assurance','transfer_owner','annual_insurance',
                'compulsory_insurance','business_insurance','price_bn'],axis=1)
    cur_time = datetime.datetime.now()
    for i in range(0, len(insert_car_source)):
        try:
            url = insert_car_source.loc[i, 'url']
            source = insert_car_source.loc[i:i,:]
            sql = 'delete from car_source where url=\'' +url+ '\' '
            con.execute(sql)
            con.execute('commit')
            source.to_sql(name='car_source', if_exists='append', con=engine, index=False)

            sql = 'select id,thumbnail from car_source where url=\'' +url+ '\' '
            query_car_source = pd.read_sql_query(sql, engine)
            final_car_source = car_source.loc[i:i,:].merge(query_car_source,how='left',on=['thumbnail'])
            car_image = pd.DataFrame()
            car_image['image'] = pd.Series(eval(final_car_source.loc[0,'car_image']))
            car_id = final_car_source.loc[0,'id']
            car_image['car_id'] = car_id
            car_image['create_time'] = cur_time
            sql = 'delete from car_image where car_id = ' + str(car_id)
            con.execute(sql)
            con.execute('commit')
            car_image.to_sql(name='car_image', if_exists='append', con=engine, index=False)

            car_detail_info = final_car_source.copy()
            car_detail_info = car_detail_info.loc[:,['conditions','description','contact','region','company_name','transfer_owner','maintenance','insurance_money','quality_assurance','id','price_bn']]
            car_detail_info['update_time'] = cur_time.strftime('%Y-%m-%d %H:%M:%S')
            car_detail_info['condition_score'] = ''
            car_detail_info = car_detail_info.rename(columns={'id':'car_id','annual_insurance':'examine_insurance','compulsory_insurance':'mandatory_insurance',
                                                             'conditions':'condition'})
            car_detail_info['mdn_status'] = 'P'
            car_detail_info['dealer_status'] = 0

            sql = 'delete from car_detail_info where car_id = ' + str(car_id)
            con.execute(sql)
            con.execute('commit')
            car_detail_info.to_sql(name='car_detail_info', if_exists='append', con=engine, index=False)
            print i
        except Exception as e:
            print e.message
con.close()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42858 entries, 0 to 42857
Data columns (total 50 columns):
source_id               42858 non-null int64
url                     42858 non-null object
title                   42858 non-null object
year                    42858 non-null int64
month                   42855 non-null float64
mile                    42858 non-null float64
city                    42858 non-null object
color                   0 non-null float64
phone                   42712 non-null object
thumbnail               42858 non-null object
price                   42858 non-null float64
domain                  42858 non-null object
transfer_owner          0 non-null float64
annual_insurance        22824 non-null object
compulsory_insurance    0 non-null float64
business_insurance      21880 non-null object
description             41445 non-null object
conditions              0 non-null float64
contact                 42858 non-null object
region                  3638

In [69]:
car_source = pd.read_csv('./finish_match.csv',low_memory=False) 

def cal_count(df):
    return len(df['image'])
car_image = pd.DataFrame()
car_image['image'] = pd.Series(eval(car_source.loc[0,'car_image']))
car_image['count'] = car_image.apply(cal_count,axis=1)
car_image

,image,count
0,http://gongpingjia.qiniudn.com/remote/5d7b08d2...,70
1,https://pic3.58cdn.com.cn/mobile/big/n_v2a4020...,77
2,https://pic4.58cdn.com.cn/mobile/big/n_v29ccc0...,77
3,https://pic5.58cdn.com.cn/mobile/big/n_v2b88c5...,77
4,https://pic5.58cdn.com.cn/mobile/big/n_v2bffe6...,77
5,https://pic8.58cdn.com.cn/mobile/big/n_v2729ca...,77
6,https://pic5.58cdn.com.cn/mobile/big/n_v26841e...,77
7,https://pic8.58cdn.com.cn/mobile/big/n_v23bc89...,77


In [36]:
car_source.to_csv('./man.csv',index=False)

In [55]:
text = '[{"category": "qiniu", "images": [{"image": "http://gongpingjia.qiniudn.com/remote/43d4589db0c492741805186f36fdf4e3", "thumb": "http://gongpingjia.qiniudn.com/remote/43d4589db0c492741805186f36fdf4e3"}]}, {"category": "图片", "images": [{"image": "http://img4.baixing.net/fa2734582aabd002189dda13adda09a8.jpg_biwbp http://img4.baixing.net/fa2734582aabd002189dda13adda09a8.jpg_biwbp http://img5.baixing.net/a2bd0f5917aa1535a0958ef73498af9b.jpg_biwbp http://img6.baixing.net/b57cf1eb1c320711af5a38378a9e8ea3.jpg_biwbp http://img4.baixing.net/0caa0babbcde7a5e8bc92fb7f7ffd8a5.jpg_biwbp http://img4.baixing.net/6b0f250df0ac618d28c85e7a4b4c7cdd.jpg_biwbp http://img6.baixing.net/5cb257dcef463fdec55f582983b2bf2c.jpg_biwbp http://img6.baixing.net/261a39fe607472a7f300ac37f2cd7aa8.jpg_biwbp http://img4.baixing.net/3313c1098d6cbfce2d93e80938d55b80.jpg_biwbp http://img5.baixing.net/d041da486331f90173b9ab0c3c00b27f.jpg_biwbp http://img5.baixing.net/d0d33f205d6c116d6de9c29eeec0b981.jpg_biwbp", "thumb": "http://img4.baixing.net/fa2734582aabd002189dda13adda09a8.jpg_biwbp http://img4.baixing.net/fa2734582aabd002189dda13adda09a8.jpg_biwbp http://img5.baixing.net/a2bd0f5917aa1535a0958ef73498af9b.jpg_biwbp http://img6.baixing.net/b57cf1eb1c320711af5a38378a9e8ea3.jpg_biwbp http://img4.baixing.net/0caa0babbcde7a5e8bc92fb7f7ffd8a5.jpg_biwbp http://img4.baixing.net/6b0f250df0ac618d28c85e7a4b4c7cdd.jpg_biwbp http://img6.baixing.net/5cb257dcef463fdec55f582983b2bf2c.jpg_biwbp http://img6.baixing.net/261a39fe607472a7f300ac37f2cd7aa8.jpg_biwbp http://img4.baixing.net/3313c1098d6cbfce2d93e80938d55b80.jpg_biwbp http://img5.baixing.net/d041da486331f90173b9ab0c3c00b27f.jpg_biwbp http://img5.baixing.net/d0d33f205d6c116d6de9c29eeec0b981.jpg_biwbp"}]}]'
thumbnail = json.loads(text)
thumbnail

[{'category': 'qiniu',
  'images': [{'image': 'http://gongpingjia.qiniudn.com/remote/43d4589db0c492741805186f36fdf4e3',
    'thumb': 'http://gongpingjia.qiniudn.com/remote/43d4589db0c492741805186f36fdf4e3'}]},
 {'category': '图片',
  'images': [{'image': 'http://img4.baixing.net/fa2734582aabd002189dda13adda09a8.jpg_biwbp http://img4.baixing.net/fa2734582aabd002189dda13adda09a8.jpg_biwbp http://img5.baixing.net/a2bd0f5917aa1535a0958ef73498af9b.jpg_biwbp http://img6.baixing.net/b57cf1eb1c320711af5a38378a9e8ea3.jpg_biwbp http://img4.baixing.net/0caa0babbcde7a5e8bc92fb7f7ffd8a5.jpg_biwbp http://img4.baixing.net/6b0f250df0ac618d28c85e7a4b4c7cdd.jpg_biwbp http://img6.baixing.net/5cb257dcef463fdec55f582983b2bf2c.jpg_biwbp http://img6.baixing.net/261a39fe607472a7f300ac37f2cd7aa8.jpg_biwbp http://img4.baixing.net/3313c1098d6cbfce2d93e80938d55b80.jpg_biwbp http://img5.baixing.net/d041da486331f90173b9ab0c3c00b27f.jpg_biwbp http://img5.baixing.net/d0d33f205d6c116d6de9c29eeec0b981.jpg_biwbp',
    'th

In [50]:
url = []
url.append(thumbnail[0]['images'][0]['image'])
url.extend(thumbnail[1]['images'][0]['image'].split(' '))
url

['http://gongpingjia.qiniudn.com/remote/536cd371ac1b077260e99a0edd63fdcf',
 'https://pic4.58cdn.com.cn/mobile/big/n_v2a754ca69c23e49a986c7e1272d566848.jpg',
 'https://pic6.58cdn.com.cn/mobile/big/n_v2f004dc26bd1942419f8c649c24d49e51.jpg',
 'https://pic6.58cdn.com.cn/mobile/big/n_v2ff0ab494000b499e878998ce6cb82ddc.jpg',
 'https://pic8.58cdn.com.cn/mobile/big/n_v2fc75d61aa04c4d619260ef580c3c59ee.jpg',
 'https://pic8.58cdn.com.cn/mobile/big/n_v2599c733241ac4ee8a6b7afcba76c9dbf.jpg',
 'https://pic7.58cdn.com.cn/mobile/big/n_v219860c4659fe45df9c3287d666c95904.jpg',
 'https://pic8.58cdn.com.cn/mobile/big/n_v25b38abdf605646a9883f2424d98fc3cc.jpg',
 'https://pic2.58cdn.com.cn/mobile/big/n_v2266ee2d546b04669808b12d32a016860.jpg',
 'https://pic8.58cdn.com.cn/mobile/big/n_v2cbde63a8a1ca4e82a9be0ea6fb6639fd.jpg']

### 二.估值数据库更新脚本

In [2]:
# coding=utf-8
import os
import re
import sys
import json
import math
import random
import datetime
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

reload(sys)
sys.setdefaultencoding('utf8')

ENCODING = 'utf-8'
PRODUCE_DB_ADDR_OUTTER = '10.45.138.200'
PRODUCE_DB_USER = 'pingjia'
PRODUCE_DB_PASSWD = 'De32wsxC'
PRODUCE_USED_CAR_ESTIMATE_ENGINE = 'mysql+pymysql://'+PRODUCE_DB_USER+':'+PRODUCE_DB_PASSWD+'@'+PRODUCE_DB_ADDR_OUTTER+'/china_used_car_estimate?charset=utf8'
engine = create_engine(PRODUCE_USED_CAR_ESTIMATE_ENGINE, encoding=ENCODING)  

global_model_mean = pd.read_csv('./global_model_mean.csv')
global_model_mean = global_model_mean.drop(['listed_year'], axis=1)

with engine.begin() as con:
    sql = 'delete from valuate_global_model_mean where update_time < \''+  datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") +'\' ' 
    con.execute(sql)
    con.execute('commit')
con.close()

global_model_mean.to_sql(name='valuate_global_model_mean', if_exists='append', con=engine, index=False)

'2019.1'

In [14]:
def process_price_bn(df):
    if df['price_bn'] == '暂无报价':
        return np.NAN
    temp = re.findall(r'万',df['price_bn'])
    if len(temp) > 1:
        text = df['price_bn'].split('~')[1]
    else:
        text = df['price_bn']
    text = text.replace('万','')
    return float(text)

def delete_str_useless(df, column_name):
    """
    删除没用的字符
    """    
    text = df[column_name]
    text = text.replace('·', '')
    text = text.replace('・', '')
    text = text.replace('-', '')
    text = text.replace(' ', '')
    text = text.lower()
    return text

# 判断是否有重复final_text
origin_train = pd.read_csv('./origin_train.csv')
origin_train = origin_train.loc[(origin_train['domain'] == 'baixing.com'),:].reset_index(drop=True)

origin_train = origin_train.loc[(origin_train['source_id'].notnull())&(origin_train['detail_name'].notnull())&(origin_train['price'] > 0)&(origin_train['transfer_owner'] == 0), :].reset_index(drop=True)
origin_train['title'] = origin_train.apply(delete_str_useless, args=('detail_name',), axis=1)
origin_train = origin_train.rename(columns={'detail_name':'origin_name'})
train_final = pd.read_csv('../tmp/train/train_final.csv')
train_final = train_final.rename(columns={'final_text':'title'})

ValueError: Cannot set a frame with no defined index and a value that cannot be converted to a Series

In [7]:
# data = wait_predict.loc[(wait_predict['domain'].isin(['guazi.com'])), :]
data = origin_train.copy()
print('瓜子车源数据量:', len(data))
final = data.merge(train_final, how='left', on=['title'])
final = final.sort_values(by=['title'])
match = final.loc[(final['online_year'].notnull())&(final['city'].notnull()), :].reset_index(drop=True)
print('瓜子车源数据匹配量:',len(match), '匹配率:', len(match)/len(data))
match.info()
match.to_csv('/home/ml/ProgramProject/evaluation-predict/tmp/train/car_source_match.csv', index=False)

瓜子车源数据量: 130718
瓜子车源数据匹配量: 122362 匹配率: 0.9360761333557734
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122362 entries, 0 to 122361
Data columns (total 18 columns):
source_id         122362 non-null int64
origin_name       122362 non-null object
year              122362 non-null int64
month             122362 non-null float64
mile              122362 non-null float64
price             122362 non-null float64
city              122362 non-null object
domain            122362 non-null object
transfer_owner    122362 non-null float64
title             122362 non-null object
brand_slug        122362 non-null float64
model_slug        122362 non-null float64
detail_slug       122362 non-null float64
online_year       122362 non-null float64
price_bn          122362 non-null float64
brand_name        122362 non-null object
model_name        122362 non-null object
detail_name       122362 non-null object
dtypes: float64(9), int64(2), object(7)
memory usage: 16.8+ MB


In [20]:
def delete_str_useless(df, column_name):
    """
    删除没用的字符
    """    
    text = df[column_name]
    text = text.replace('·', '')
    text = text.replace('・', '')
    text = text.replace('-', '')
    text = text.replace(' ', '')
    text = text.lower()
    return text

# 判断是否有重复final_text
origin_train = pd.read_csv('./origin_train.csv')
origin_train = origin_train.loc[(origin_train['domain'] == 'xin.com'),:].reset_index(drop=True)

origin_train = origin_train.loc[(origin_train['source_id'].notnull())&(origin_train['detail_name'].notnull())&(origin_train['price'] > 0), :].reset_index(drop=True)
origin_train['title'] = origin_train.apply(delete_str_useless, args=('detail_name',), axis=1)
origin_train = origin_train.rename(columns={'detail_name':'origin_name'})
train_final = pd.read_csv('../tmp/train/train_final.csv')
train_final = train_final.rename(columns={'final_text':'title'})

In [21]:
# data = wait_predict.loc[(wait_predict['domain'].isin(['guazi.com'])), :]
data = origin_train.copy()
print('优信车源数据量:', len(data))
final = data.merge(train_final, how='left', on=['title'])
final = final.sort_values(by=['title'])
match = final.loc[(final['online_year'].notnull())&(final['city'].notnull()), :].reset_index(drop=True)
print('优信车源数据匹配量:',len(match), '匹配率:', len(match)/len(data))
match.info()
match.to_csv('/home/ml/ProgramProject/evaluation-predict/tmp/train/car_source_match.csv', index=False)

百姓车源数据量: 3115
百姓车源数据匹配量: 60 匹配率: 0.019261637239165328
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 18 columns):
source_id         60 non-null int64
origin_name       60 non-null object
year              60 non-null int64
month             0 non-null float64
mile              60 non-null float64
price             60 non-null float64
city              60 non-null object
domain            60 non-null object
transfer_owner    0 non-null float64
title             60 non-null object
brand_slug        60 non-null float64
model_slug        60 non-null float64
detail_slug       60 non-null float64
online_year       60 non-null float64
price_bn          60 non-null float64
brand_name        60 non-null object
model_name        60 non-null object
detail_name       60 non-null object
dtypes: float64(9), int64(2), object(7)
memory usage: 8.5+ KB


In [23]:
origin_train

,source_id,origin_name,year,month,mile,price,city,domain,transfer_owner,title
0,29001395,奔腾 B50 2013款 1.6 手动 舒适型,2015,NaN,5.8,4.30,淄博,xin.com,NaN,奔腾b502013款1.6手动舒适型
1,44451994,丰田 威驰 2017款 1.5 自动 创行版,2017,NaN,1.9,6.98,宜昌,xin.com,NaN,丰田威驰2017款1.5自动创行版
2,25775446,铃木 奥拓 2013款 1.0 手动 豪华型,2015,NaN,2.7,2.80,宜昌,xin.com,NaN,铃木奥拓2013款1.0手动豪华型
3,59270101,东南 V3菱悦 2015款 1.5 手动 幸福版,2016,NaN,3.9,3.50,宁波,xin.com,NaN,东南v3菱悦2015款1.5手动幸福版
4,31517452,丰田 致享 2017款 1.5 自动 E魅动版,2018,NaN,0.6,7.38,大连,xin.com,NaN,丰田致享2017款1.5自动e魅动版
5,37330160,大众 朗逸三厢 2010款 1.6 自动 品雅进享版,2010,NaN,7.9,4.50,银川,xin.com,NaN,大众朗逸三厢2010款1.6自动品雅进享版
6,69042933,雪佛兰 科鲁兹三厢 2009款 1.6 手动 SE,2011,NaN,14.3,4.18,宁波,xin.com,NaN,雪佛兰科鲁兹三厢2009款1.6手动se
7,50072905,本田 雅阁 2018款 1.5T 自动 260TURBO精英版,2019,NaN,0.1,18.20,承德,xin.com,NaN,本田雅阁2018款1.5t自动260turbo精英版
8,43200098,现代 瑞纳三厢 2014款 1.4 手动 GLS智能型,2016,NaN,2.0,4.60,银川,xin.com,NaN,现代瑞纳三厢2014款1.4手动gls智能型
9,51615566,日产 骐达 2011款 1.6 自动 智能型,2012,NaN,10.2,5.70,洛阳,xin.com,NaN,日产骐达2011款1.6自动智能型


In [ ]:
#----------------
# 1.车系去掉品牌
# 2.1.5L去掉L